In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from IPython.display import display, HTML

import calendar
import datetime as dt

import re

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/saleforcasting/datos/sales_multistore_10_2019_04_2020.csv',nrows= 100000)

In [ ]:
df.head()

,event_time,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:02:14 UTC,1004856,2053013555631882655,electronics.smartphone,samsung,130.76,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564
1,2019-10-01 00:04:37 UTC,1002532,2053013555631882655,electronics.smartphone,apple,642.69,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68
2,2019-10-01 00:07:07 UTC,13800054,2053013557418656265,furniture.bathroom.toilet,santeri,54.42,555332717,1dea3ee2-2ded-42e8-8e7a-4e2ad6ae942f
3,2019-10-01 00:09:26 UTC,4804055,2053013554658804075,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553
4,2019-10-01 00:09:54 UTC,4804056,2053013554658804075,electronics.audio.headphone,apple,161.98,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68


In [ ]:
df.shape

(100000, 8)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     100000 non-null  object 
 1   product_id     100000 non-null  int64  
 2   category_id    100000 non-null  int64  
 3   category_code  100000 non-null  object 
 4   brand          100000 non-null  object 
 5   price          100000 non-null  float64
 6   user_id        100000 non-null  int64  
 7   user_session   100000 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 6.1+ MB


In [ ]:
df["user_id"] = df["user_id"].fillna(0)
df["user_id"] = df["user_id"].astype(int)
df["user_id"] = df["user_id"].astype(str)

In [ ]:
df["quantity"] = df["price"].apply(lambda x:1)

In [ ]:
df.head()

,event_time,product_id,category_id,category_code,brand,price,user_id,user_session,quantity
0,2019-10-01 00:02:14 UTC,1004856,2053013555631882655,electronics.smartphone,samsung,130.76,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564,1
1,2019-10-01 00:04:37 UTC,1002532,2053013555631882655,electronics.smartphone,apple,642.69,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68,1
2,2019-10-01 00:07:07 UTC,13800054,2053013557418656265,furniture.bathroom.toilet,santeri,54.42,555332717,1dea3ee2-2ded-42e8-8e7a-4e2ad6ae942f,1
3,2019-10-01 00:09:26 UTC,4804055,2053013554658804075,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553,1
4,2019-10-01 00:09:54 UTC,4804056,2053013554658804075,electronics.audio.headphone,apple,161.98,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68,1


In [ ]:
df= df.dropna()

In [ ]:
df['event_time']=pd.to_datetime(df['event_time'])

In [ ]:

df['Date'] = df['event_time'].dt.date
df['Month'] = df['event_time'].dt.month
df['Year'] = df['event_time'].dt.year
df['Month'] = df['Month'].apply(lambda x: calendar.month_name[int(x)])
df['Week of the Year'] = df['event_time'].dt.isocalendar().week
df['Day of Week'] = df['event_time'].dt.day_name()
df['Time'] = df['event_time'].dt.time
df["Hour"] = df['event_time'].dt.hour
df['Time of Day']=pd.cut(df['event_time'].dt.hour,[0,6,12,18,23],labels=['Night','Morning','Afternoon','Evening'],include_lowest=True)

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
df['user_id'] = df['user_id'].astype('int64')

In [ ]:
basket = (df.groupby(['brand', 'category_code'])['quantity'].sum().unstack().reset_index().fillna(0).set_index('brand'))

In [ ]:
basket.head()

category_code,accessories.bag,accessories.umbrella,accessories.wallet,apparel.scarf,apparel.shirt,apparel.shoes,apparel.shoes.keds,apparel.shoes.moccasins,apparel.shoes.slipons,apparel.trousers,...,kids.fmcg.diapers,kids.skates,kids.swing,kids.toys,medicine.tools.tonometer,sport.bicycle,sport.ski,sport.snowboard,sport.trainer,stationery.cartrige
brand,,,,,,,,,,,,,,,,,,,,,
acer,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
acme,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
acv,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adagio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adamex,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#basket.dropna()

In [ ]:
def encoder(x):
  if(x <= 0):
    return 0
  if(x >= 1):
    return 1


basket = basket.applymap(encoder)
basket.head()

category_code,accessories.bag,accessories.umbrella,accessories.wallet,apparel.scarf,apparel.shirt,apparel.shoes,apparel.shoes.keds,apparel.shoes.moccasins,apparel.shoes.slipons,apparel.trousers,...,kids.fmcg.diapers,kids.skates,kids.swing,kids.toys,medicine.tools.tonometer,sport.bicycle,sport.ski,sport.snowboard,sport.trainer,stationery.cartrige
brand,,,,,,,,,,,,,,,,,,,,,
acer,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
acme,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
acv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
adagio,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
adamex,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#basket= basket.astype(bool)

In [ ]:
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.034639,(accessories.bag),1
1,0.042169,(apparel.shoes),1
2,0.037651,(apparel.shoes.keds),1
3,0.019578,(appliances.environment.air_conditioner),1
4,0.028614,(appliances.environment.air_heater),1
...,...,...,...
760,0.010542,"(appliances.kitchen.oven, appliances.kitchen.w...",6
761,0.010542,"(appliances.kitchen.mixer, appliances.kitchen....",7
762,0.010542,"(appliances.kitchen.mixer, appliances.kitchen....",7
763,0.010542,"(appliances.kitchen.grill, appliances.kitchen....",7


In [ ]:
frequent_itemsets.max()

support                                              0.072289
itemsets    (appliances.kitchen.mixer, appliances.kitchen....
length                                                      7
dtype: object

In [ ]:
rules = association_rules(frequent_itemsets, metric = 'lift', min_threshold = 0.0001)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(accessories.bag),(computers.peripherals.mouse),0.034639,0.034639,0.010542,0.304348,8.786389,0.009342,1.387707
1,(computers.peripherals.mouse),(accessories.bag),0.034639,0.034639,0.010542,0.304348,8.786389,0.009342,1.387707
2,(accessories.bag),(electronics.audio.headphone),0.034639,0.072289,0.010542,0.304348,4.210145,0.008038,1.333584
3,(electronics.audio.headphone),(accessories.bag),0.072289,0.034639,0.010542,0.145833,4.210145,0.008038,1.130179
4,(apparel.shoes),(apparel.shoes.keds),0.042169,0.037651,0.016566,0.392857,10.434286,0.014979,1.585046
...,...,...,...,...,...,...,...,...,...
10469,(appliances.kitchen.coffee_machine),"(appliances.kitchen.mixer, appliances.kitchen....",0.021084,0.010542,0.010542,0.500000,47.428571,0.010320,1.978916
10470,(appliances.kitchen.blender),"(appliances.kitchen.mixer, appliances.kitchen....",0.034639,0.010542,0.010542,0.304348,28.869565,0.010177,1.422346
10471,(appliances.kitchen.kettle),"(appliances.kitchen.mixer, appliances.kitchen....",0.048193,0.010542,0.010542,0.218750,20.750000,0.010034,1.266506
10472,(appliances.kitchen.juicer),"(appliances.kitchen.mixer, appliances.kitchen....",0.021084,0.012048,0.010542,0.500000,41.500000,0.010288,1.975904


In [ ]:
rules = rules.sort_values("lift",ascending=False).reset_index(drop= True)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(appliances.kitchen.oven, appliances.kitchen.r...","(appliances.kitchen.microwave, appliances.kitc...",0.010542,0.012048,0.010542,1.000,83.0,0.010415,inf
1,"(appliances.kitchen.toster, appliances.kitchen...","(appliances.iron, appliances.kitchen.grill, ap...",0.012048,0.010542,0.010542,0.875,83.0,0.010415,7.915663
2,"(appliances.kitchen.microwave, appliances.kitc...","(appliances.kitchen.dishwasher, appliances.kit...",0.010542,0.012048,0.010542,1.000,83.0,0.010415,inf
3,"(appliances.kitchen.coffee_grinder, appliances...","(appliances.kitchen.grill, appliances.kitchen....",0.012048,0.010542,0.010542,0.875,83.0,0.010415,7.915663
4,"(appliances.kitchen.coffee_grinder, appliances...","(appliances.kitchen.grill, appliances.kitchen....",0.012048,0.010542,0.010542,0.875,83.0,0.010415,7.915663
